<a href="https://colab.research.google.com/github/huxd2334/Data-Structures-and-Algorithms-Lab/blob/master/Get_Sentinel_1_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings("ignore")

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Planetary Computer Tools
!pip install planetary_computer odc-stac pystac pystac_client
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc

# API key
pc.settings.set_subscription_key("st=2024-10-30T04%3A39%3A47Z&se=2024-10-31T05%3A24%3A47Z&sp=rl&sv=2024-05-04&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2024-10-31T01%3A20%3A26Z&ske=2024-11-07T01%3A20%3A26Z&sks=b&skv=2024-05-04&sig=yxW50NrgqfNEwh6gA5GpPmjbepQ0PP8d0LIG7B5GgAU%3D")

# Other
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()
import textwrap
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
!pip install stackstac
import stackstac
!pip install statistics
import statistics


In [15]:
locations = pd.read_csv('/content/drive/MyDrive/Crop_Yield_Data.csv')

In [3]:
import pandas as pd
from datetime import datetime, timedelta

In [18]:
def extract_sar_data(locations):
    A = 0.71
    B = 1.40
    all_vv = []
    all_vh = []
    all_rvi = []
    all_soil_moisture = []

    for index, row in locations.iterrows():
        print(f"Processing row {index + 1}")
        lon = row['Longitude']
        lat = row['Latitude']
        toh = row['Date of Harvest']
        date_object = datetime.strptime(toh, "%d-%m-%Y")

        time_delta = timedelta(days=6)
        start_date = date_object - time_delta
        end_date = date_object + time_delta
        time_window3 = f"{start_date.isoformat()}/{end_date.isoformat()}"

        box_size_deg2 = 0.0004  # ~ 5x5 px
        resolution = 10  # meters per px
        scale2 = resolution / 111320.0  # degrees per px (for crs=4326)
        bbox = (lon - box_size_deg2/2, lat - box_size_deg2/2,
               lon + box_size_deg2/2, lat + box_size_deg2/2)

        try:
            catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
            search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_window3)
            items = list(search.get_all_items())
            data = stac_load(items, bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox, crs="EPSG:4326", resolution=scale2)

            # Extract VV and VH arrays
            vv_array = data.vv.compute().values
            vh_array = data.vh.compute().values

            # Calculate RVI array
            rvi_array = (4 * vh_array / (vv_array + vh_array))
            print(f"RVI Array Shape: {rvi_array}")
            # Calculate DOP and soil moisture arrays
            dop_array = vv_array / (vv_array + vh_array)
            soil_moisture_array = 1 - ((10 ** (0.1 * dop_array)) / A) ** B
            print(f"Soil Moisture Array Shape: {soil_moisture_array}")

            # Store arrays
            all_vv.append(vv_array)
            all_vh.append(vh_array)
            all_rvi.append(rvi_array)
            all_soil_moisture.append(soil_moisture_array)



        except Exception as e:
            print(f"Error processing location {index}: {str(e)}")
            continue

    return all_vv, all_vh, all_rvi, all_soil_moisture



In [19]:
vv_arrays, vh_arrays, rvi_arrays, soil_moisture_arrays = extract_sar_data(locations)

Streaming output truncated to the last 5000 lines.
  [-1.0661035  -1.0935183  -1.11411    -1.1367393  -1.1426508 ]
  [-1.0511479  -1.0292735  -1.047327   -1.0906632  -1.1203938 ]
  [-1.0830157  -1.0726235  -1.0965667  -1.1445448  -1.1632094 ]
  [-1.0742302  -1.0915189  -1.1410913  -1.1759846  -1.1618578 ]]

 [[-0.91402876 -0.97828054 -0.935892   -0.86640775 -0.89413893]
  [-0.91402876 -0.90246594 -0.92952645 -0.98076105 -0.9986373 ]
  [-0.9083078  -0.93185294 -0.90868783 -0.9374349  -1.0100765 ]
  [-0.91542697 -0.9518242  -0.935693   -0.9386476  -0.9445751 ]
  [-1.099133   -1.0415812  -1.0169542  -0.96942985 -0.91421115]
  [-1.1672461  -1.1199467  -1.044246   -0.98098505 -1.011275  ]]]
Processing row 343
RVI Array Shape: [[[1.7730494  1.6520156  1.5528203  1.6206787  1.8821173 ]
  [1.3288074  1.3876506  1.7510283  2.332374   2.5375898 ]
  [0.74184674 1.2632548  1.4866455  1.687826   1.9120153 ]
  [0.6829131  1.7331536  1.622419   0.9430335  0.96908027]
  [0.8414491  1.597701   2.327837

In [20]:
def calculate_sar_statistics(vv_arrays, vh_arrays, rvi_arrays, soil_moisture_arrays):
    from sklearn.preprocessing import MinMaxScaler

    stats = []

    for i in range(len(vv_arrays)):
        # Basic statistics for VV, VH, and RVI
        stats_dict = {
            'vv_mean': np.nanmean(vv_arrays[i]),
            'vh_mean': np.nanmean(vh_arrays[i])
        }
         # Handle RVI scaling
        rvi_array = rvi_arrays[i]
        stats_dict.update({
            'rvi_mean_original': np.nanmean(rvi_array),
            'rvi_original_min': np.nanmin(rvi_array),
            'rvi_original_max': np.nanmax(rvi_array),
        })

        # Prepare RVI data for scaling
        rvi_flat = rvi_array.flatten()
        rvi_flat = rvi_flat[~np.isnan(rvi_flat)]  # Remove NaN values
        rvi_flat = rvi_flat.reshape(-1, 1)  # Reshape for scaler

        # Scale RVI between 0 and 1
        rvi_scaler = MinMaxScaler(feature_range=(0, 1))
        try:
            rvi_scaled = rvi_scaler.fit_transform(rvi_flat)

            # Add scaled RVI statistics
            stats_dict.update({
                'rvi_scaled_mean': np.mean(rvi_scaled),
                'rvi_scaled_min': np.min(rvi_scaled),
                'rvi_scaled_max': np.max(rvi_scaled),
            })
        except Exception as e:
            print(f"Warning: Could not scale RVI for index {i}: {str(e)}")
            stats_dict.update({
                'rvi_scaled_mean': np.nan,
                'rvi_scaled_min': np.nan,
                'rvi_scaled_max': np.nan,
            })

        # Handle soil moisture scaling
        sm_array = soil_moisture_arrays[i]

        stats_dict.update({
            'soil_moisture_original_mean': np.nanmean(sm_array)
        })

        # Prepare soil moisture data for scaling
        sm_flat = sm_array.flatten()
        sm_flat = sm_flat[~np.isnan(sm_flat)]  # Remove NaN values
        sm_flat = sm_flat.reshape(-1, 1)  # Reshape for scaler

        # Scale soil moisture between 0 and 1
        scaler = MinMaxScaler(feature_range=(0, 1))
        try:
            sm_scaled = scaler.fit_transform(sm_flat)

            # Add scaled statistics
            stats_dict.update({
                'soil_moisture_scaled_mean': np.mean(sm_scaled)
            })
        except Exception as e:
            print(f"Warning: Could not scale soil moisture for index {i}: {str(e)}")
            stats_dict.update({
                'soil_moisture_scaled_mean': np.nan
            })

        stats.append(stats_dict)

    # Convert to DataFrame
    stats_df = pd.DataFrame(stats)

    return stats_df

In [21]:
stats_df = calculate_sar_statistics(vv_arrays, vh_arrays, rvi_arrays, soil_moisture_arrays)
stats_df

,vv_mean,vh_mean,rvi_mean_original,rvi_original_min,rvi_original_max,rvi_scaled_mean,rvi_scaled_min,rvi_scaled_max,soil_moisture_original_mean,soil_moisture_scaled_mean
0,0.064993,0.016542,0.816603,0.438220,1.344681,0.417429,0.0,1.0,-1.087973,0.424182
1,0.106536,0.028129,0.954783,0.330124,2.284428,0.319632,0.0,1.0,-1.066020,0.332197
2,0.067111,0.027924,1.204159,0.394232,2.056718,0.487179,0.0,1.0,-1.025342,0.496937
3,0.032327,0.027576,1.881347,0.465045,2.713330,0.629948,0.0,1.0,-0.917774,0.645850
4,0.173245,0.032640,0.771423,0.203681,1.949465,0.325207,0.0,1.0,-1.096376,0.336901
...,...,...,...,...,...,...,...,...,...,...
552,0.272681,0.054970,0.720642,0.194783,1.438500,0.422812,0.0,1.0,-1.104383,0.432551
553,0.046352,0.006637,0.595935,0.122947,1.336088,0.389887,0.0,1.0,-1.125663,0.398847
554,0.024154,0.004418,0.697654,0.234651,1.600070,0.339092,0.0,1.0,-1.108588,0.347847
555,0.120502,0.010860,0.367932,0.097001,1.048553,0.284726,0.0,1.0,-1.164761,0.291075


In [22]:
stats_df = stats_df.drop(columns=['rvi_scaled_min', 'rvi_scaled_max'])

In [23]:
stats_df

,vv_mean,vh_mean,rvi_mean_original,rvi_original_min,rvi_original_max,rvi_scaled_mean,soil_moisture_original_mean,soil_moisture_scaled_mean
0,0.064993,0.016542,0.816603,0.438220,1.344681,0.417429,-1.087973,0.424182
1,0.106536,0.028129,0.954783,0.330124,2.284428,0.319632,-1.066020,0.332197
2,0.067111,0.027924,1.204159,0.394232,2.056718,0.487179,-1.025342,0.496937
3,0.032327,0.027576,1.881347,0.465045,2.713330,0.629948,-0.917774,0.645850
4,0.173245,0.032640,0.771423,0.203681,1.949465,0.325207,-1.096376,0.336901
...,...,...,...,...,...,...,...,...
552,0.272681,0.054970,0.720642,0.194783,1.438500,0.422812,-1.104383,0.432551
553,0.046352,0.006637,0.595935,0.122947,1.336088,0.389887,-1.125663,0.398847
554,0.024154,0.004418,0.697654,0.234651,1.600070,0.339092,-1.108588,0.347847
555,0.120502,0.010860,0.367932,0.097001,1.048553,0.284726,-1.164761,0.291075


In [24]:
stats_df.to_csv('/content/drive/MyDrive/data_/sentinel-1_df.csv', index=False)